In [5]:
import argparse

from collections import OrderedDict
from pipeline.backend.pipeline import PipeLine
from pipeline.component import DataTransform
from pipeline.component import HeteroNN
from pipeline.component import Intersection
from pipeline.component import Reader
from pipeline.component import Evaluation
from pipeline.interface import Data
from pipeline.utils.tools import load_job_config, JobConfig
from pipeline.interface import Model
from federatedml.evaluation.metrics import classification_metric
from fate_test.utils import extract_data, parse_summary_result

from pipeline import fate_torch_hook
import torch as t
from torch import nn
from torch.nn import init
from torch import optim
from pipeline import fate_torch as ft

fate_torch_hook(t)

<module 'torch' from '/data/projects/fate/common/python/venv/lib/python3.8/site-packages/torch/__init__.py'>

In [20]:
def main(config="./config.yaml", param="./hetero_nn_breast_config.yaml", namespace=""):
    # obtain config
    if isinstance(config, str):
        config = load_job_config(config)

    if isinstance(param, str):
        param = JobConfig.load_from_file(param)

    parties = config.parties
    guest = parties.guest[0]
    host = parties.host[0]

    guest_train_data = {"name": param["guest_table_name"], "namespace": f"experiment{namespace}"}
    host_train_data = {"name": param["host_table_name"], "namespace": f"experiment{namespace}"}

    pipeline = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest, host=host)

    reader_0 = Reader(name="reader_0")
    reader_0.get_party_instance(role='guest', party_id=guest).component_param(table=guest_train_data)
    reader_0.get_party_instance(role='host', party_id=host).component_param(table=host_train_data)

    data_transform_0 = DataTransform(name="data_transform_0")
    data_transform_0.get_party_instance(role='guest', party_id=guest).component_param(with_label=True)
    data_transform_0.get_party_instance(role='host', party_id=host).component_param(with_label=False)

    intersection_0 = Intersection(name="intersection_0")
    
    guest_input_shape = param['guest_input_shape']
    host_input_shape = param['host_input_shape']   
    # define model structures
    bottom_model_guest = t.nn.Sequential(
        nn.Linear(guest_input_shape, 10, True),
        nn.ReLU(),
        nn.Linear(10, 8, True),
        nn.ReLU()
    )
    
    bottom_model_host = t.nn.Sequential(
        nn.Linear(host_input_shape, 10, True),
        nn.ReLU(),
        nn.Linear(10, 8, True),
        nn.ReLU()
    )
    
    interactive_layer = t.nn.Linear(8, 4, True)
    
    top_model_guest = t.nn.Sequential(
        nn.Linear(4, 1, True),
        nn.Sigmoid()
    )
    
    loss_fn = nn.BCELoss()
    
    opt: ft.optim.Adam = optim.Adam(lr=param['learning_rate'])
    
    hetero_nn_0 = HeteroNN(name="hetero_nn_0", epochs=param["epochs"],
                       interactive_layer_lr=param["learning_rate"], batch_size=param["batch_size"],
                       early_stop="diff")
    guest_nn_0 = hetero_nn_0.get_party_instance(role='guest', party_id=guest)
    guest_nn_0.add_bottom_model(bottom_model_guest)
    guest_nn_0.add_top_model(top_model_guest)
    guest_nn_0.set_interactve_layer(interactive_layer)
    host_nn_0 = hetero_nn_0.get_party_instance(role='host', party_id=host)
    host_nn_0.add_bottom_model(bottom_model_host)
    # do remember to compile
    hetero_nn_0.compile(opt, loss=loss_fn)
    
    hetero_nn_1 = HeteroNN(name="hetero_nn_1")

    evaluation_0 = Evaluation(name="evaluation_0", eval_type=param['eval_type'])

    pipeline.add_component(reader_0)
    pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
    pipeline.add_component(intersection_0, data=Data(data=data_transform_0.output.data))
    pipeline.add_component(hetero_nn_0, data=Data(train_data=intersection_0.output.data))
    pipeline.add_component(hetero_nn_1, data=Data(test_data=intersection_0.output.data),
                           model=Model(hetero_nn_0.output.model))
    pipeline.add_component(evaluation_0, data=Data(data=hetero_nn_0.output.data))

    pipeline.compile()

    pipeline.fit()
    
    nn_0_data = pipeline.get_component("hetero_nn_0").get_output_data()
    nn_1_data = pipeline.get_component("hetero_nn_1").get_output_data()
    nn_0_score = extract_data(nn_0_data, "predict_result")
    nn_0_label = extract_data(nn_0_data, "label")
    nn_1_score = extract_data(nn_1_data, "predict_result")
    nn_1_label = extract_data(nn_1_data, "label")
    nn_0_score_label = extract_data(nn_0_data, "predict_result", keep_id=True)
    nn_1_score_label = extract_data(nn_1_data, "predict_result", keep_id=True)
    metric_summary = parse_summary_result(pipeline.get_component("evaluation_0").get_summary())
    eval_type = param['eval_type']
    if eval_type == "binary":
        metric_nn = {
            "score_diversity_ratio": classification_metric.Distribution.compute(nn_0_score_label, nn_1_score_label),
            "ks_2samp": classification_metric.KSTest.compute(nn_0_score, nn_1_score),
            "mAP_D_value": classification_metric.AveragePrecisionScore().compute(nn_0_score, nn_1_score, nn_0_label,
                                                                                 nn_1_label)}
        metric_summary["distribution_metrics"] = {"hetero_nn": metric_nn}
    elif eval_type == "multi":
        metric_nn = {
            "score_diversity_ratio": classification_metric.Distribution.compute(nn_0_score_label, nn_1_score_label)}
        metric_summary["distribution_metrics"] = {"hetero_nn": metric_nn}

    data_summary = {"train": {"guest": guest_train_data["name"], "host": host_train_data["name"]},
                    "test": {"guest": guest_train_data["name"], "host": host_train_data["name"]}
                    }
    return data_summary, metric_summary

In [27]:
a, b = main(param='./hetero_nn_give_credit_config.yaml')

2022-08-25 17:59:43.769 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202208251759433487130

2022-08-25 17:59:43.775 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
m2022-08-25 17:59:45.802 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2022-08-25 17:59:45.803 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2022-08-25 17:59:46.815 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2022-08-25 17:59:47.828 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2022-08-25 17:59:48.841 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:05
2022-08-25 17:59:49.856 | INFO     

2022-08-25 18:00:20.309 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:36
2022-08-25 18:00:21.322 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:37
2022-08-25 18:00:22.335 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:38
2022-08-25 18:00:23.354 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:39
2022-08-25 18:00:24.367 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:40
2022-08-25 18:00:25.380 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:41
2022-08-25 18:00:26.394 | INFO     | pipeline.utils.invoker.job_submitter:mo

2022-08-25 18:00:58.869 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:15
2022-08-25 18:00:59.882 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:16
2022-08-25 18:01:00.895 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:17
2022-08-25 18:01:01.909 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:18
2022-08-25 18:01:02.922 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:19
2022-08-25 18:01:03.935 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:20
2022-08-25 18:01:04.948 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2022-08-25 18:01:38.399 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:54
2022-08-25 18:01:39.413 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:55
2022-08-25 18:01:40.426 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:56
2022-08-25 18:01:41.439 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:57
2022-08-25 18:01:42.451 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:58
2022-08-25 18:01:43.465 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:59
2022-08-25 18:01:44.480 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2022-08-25 18:02:17.938 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:34
2022-08-25 18:02:18.951 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:35
2022-08-25 18:02:19.964 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:36
2022-08-25 18:02:20.978 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:37
2022-08-25 18:02:21.991 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:38
2022-08-25 18:02:23.004 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:39
2022-08-25 18:02:24.017 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

...

2022-08-25 20:52:21.318 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:52:37
2022-08-25 20:52:22.331 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:52:38
2022-08-25 20:52:23.350 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:52:39
2022-08-25 20:52:24.366 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:52:40
2022-08-25 20:52:25.379 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:52:41
2022-08-25 20:52:26.395 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:52:42
2022-08-25 20:52:27.409 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2022-08-25 20:53:00.885 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:53:17
2022-08-25 20:53:01.899 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:53:18
2022-08-25 20:53:02.912 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:53:19
2022-08-25 20:53:03.926 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:53:20
2022-08-25 20:53:04.940 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:53:21
2022-08-25 20:53:05.953 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:53:22
2022-08-25 20:53:06.966 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2022-08-25 20:53:40.432 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:53:56
2022-08-25 20:53:41.445 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:53:57
2022-08-25 20:53:42.459 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:53:58
2022-08-25 20:53:43.471 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:53:59
2022-08-25 20:53:44.485 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:54:00
2022-08-25 20:53:45.498 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:54:01
2022-08-25 20:53:46.511 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2022-08-25 20:54:19.956 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:54:36
2022-08-25 20:54:20.969 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:54:37
2022-08-25 20:54:21.982 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:54:38
2022-08-25 20:54:22.995 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:54:39
2022-08-25 20:54:24.008 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:54:40
2022-08-25 20:54:25.022 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 2:54:41
2022-08-25 20:54:26.036 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2022-08-25 20:54:58.485 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 2:55:14
2022-08-25 20:54:59.503 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 2:55:15
2022-08-25 20:55:00.518 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 2:55:16
2022-08-25 20:55:01.531 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 2:55:17
2022-08-25 20:55:02.545 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 2:55:18
2022-08-25 20:55:03.558 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 2:55:19
2022-08-25 20:55:04.572 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_st